# Prediction of next word of a sentence using Quadgram Model

In [2]:
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk

### 1)Read the corpus file line by line
### 2)Preprocess the content of the file:
       (i) Lower case all the words
       (ii) Remove puntuation marks

In [3]:
tokens=[]
#with open('/home/meghana/new_jupyter/assign1/Data/Tokenization/Chat1.txt') as f:
with open('/home/meghana/nltk_data/corpora/gutenberg/corpusfile.txt') as f:

    for line in f:
        if not line.isspace():
            items=line.split()
            #print("words in list;",items)
            i=0
            for w in items:
                items[i]=w.lower()
                items[i]=items[i].replace('.','').replace(',','').replace(':','').replace(';','').replace('!','').replace('?','').replace('(','').replace(')','').replace("'",'')                                 
                i+=1
            tokens+=items
tokenSet=set(tokens)
print(len(tokens))
print(len(tokenSet))

2053640
62705


In [57]:
# filtered_words=[word for word in tokens if word not in stopwords.words('english')]
# #filtered_words_set=set(filtered_words)
# print(len(filtered_words))
# print(len(filtered_words_set))
# tokens=filtered_words
# tokenSet=set(tokens)

### Split the words of the files as ngrams

In [4]:
#%%timeit
unigrams=list(ngrams(tokens,1))
trigrams=list(ngrams(tokens,3))
trigramSet=set(trigrams)
quadgrams=list(ngrams(tokens,4))
quadgramSet=set(quadgrams)
print(len(trigramSet))
print(len(quadgramSet))

1421195
1826910


### construct the probability tables of quadgrams and trigrams

In [5]:
#%%timeit
from collections import Counter

unigram_table=Counter(unigrams)
default=(unigram_table.most_common(1))
default=str(default[0][0])
default=default.replace('(','').replace("'",'').replace(',','').replace(')','')
#print(default)

trigram_list=[]
for item in trigrams:
    new_item=' '.join(item)
    trigram_list.append(new_item)

trigram_table=Counter(trigram_list)   
#print(trigram_table.most_common(10))


quadgram_list=[]
for item in quadgrams:
    new_item=' '.join(item)
    quadgram_list.append(new_item)

quadgram_table=Counter(quadgram_list)   
#print(quadgram_table.most_common(10))
#print(quadgram_table)

### 1)Look up the quadgram and trigram tables 
### 2)Take the input string
### 3)Find out the probability and predict the next word

In [6]:
#%%timeit
#s="I am Meghana Bandaru and I must"
s=input("Enter a string:")
s_tokens=s.split()
#s_filtered_words=[word for word in s_tokens if word not in stopwords.words('english')]
#s_tokens=s_filtered_words
i=0
for w in s_tokens:
    s_tokens[i]=w.lower()
    s_tokens[i]=s_tokens[i].replace('.','').replace(',','').replace(':','').replace(';','').replace('!','').replace('?','').replace('(','').replace(')','').replace("'",'')                                 
    i+=1

s_trigrams=list(ngrams(s_tokens, 3))
#print(s_trigrams)
n=len(s_trigrams)
req_trigram=s_trigrams[n-1]
req_trigram=' '.join(req_trigram)
#print(req_trigram)

prob=0
freq_tri=0
freq_quad=0
max_prob=0
result=0
word=0
for item in tokenSet:
    new_req_trigram=req_trigram+' '+item
    freq_quad=quadgram_table[new_req_trigram]
    freq_tri=trigram_table[req_trigram]
    if(freq_tri==0):
        break
    prob=freq_quad/freq_tri
    if(prob>max_prob):
        max_prob=prob
        result=new_req_trigram
        word=item
if(word==0):
    word=default
    result=req_trigram+' '+str(default)
    
print("The next word is:", word)
#print("The sentence might be:", result)

Enter a string:I am Meghana Bandaru and I must
The next word is: say
